In [2]:
import pandas as pd
import numpy as np 
import glob
from tqdm import tqdm
import sys
import os

files = sorted(glob.glob("../../data/Targets/high_10m/*.csv.gz"))
files

[]

In [3]:
days = []

for fn in tqdm(files, desc= "Reading files"):
    day = pd.read_csv(fn, compression= "gzip", parse_dates= ["DATE"])
    days.append(day)

df = pd.concat(days, ignore_index= True)

Reading files: 0it [00:00, ?it/s]


ValueError: No objects to concatenate

In [ ]:
print("Shape of the data: ", df.shape)
print("Number of stocks: ", df["SYMBOL"].nunique())


Shape of the data:  (6156358, 6)
Number of stocks:  9509


In [ ]:
df = df.sort_values(by= ["SYMBOL", "DATE"])
df.isnull().sum()

NameError: name 'df' is not defined

### IF WE WANT TO TRADE DAILY AND NOT CARE ABOUT THE OVERNIGHT SHIFT, WE CALCULATE THE RETURN ON THE DAILY BASIS, MEANING I GROUPBY THE DATE AND WE START WITH A RETURN OF 0 EVERY DAY, WE CAN CHANGE THIS IN THE FUTURE

In [ ]:
# Calculate the return on the column MID_OPEN, for each stock "SYMBOL" and on each day "DATE"
df["RETURN_NoOVERNIGHT"] = (df.groupby(["SYMBOL", "DATE"])["MID_OPEN"].pct_change()) # Best way to calcuate the return
df["RETURN_NoOVERNIGHT"] = df["RETURN_NoOVERNIGHT"].fillna(0)

df["LOG_RETURN_NoOVERNIGHT"] = (df.groupby(["SYMBOL", "DATE"])["MID_OPEN"].apply(lambda x: np.log(x / x.shift(1))))
df["LOG_RETURN_NoOVERNIGHT"] = df["LOG_RETURN_NoOVERNIGHT"].fillna(0)

In [ ]:
# Calculate the return on the column MID_OPEN, for each stock "SYMBOL" and on each day "DATE"
df["RETURN_SiOVERNIGHT"] = (df.groupby("SYMBOL")["MID_OPEN"].pct_change()) # Best way to calcuate the return
df["RETURN_SiOVERNIGHT"] = df["RETURN_SiOVERNIGHT"].fillna(0)

df["LOG_RETURN_SiOVERNIGHT"] = (df.groupby("SYMBOL")["MID_OPEN"].apply(lambda x: np.log(x / x.shift(1))))
df["LOG_RETURN_SiOVERNIGHT"] = df["LOG_RETURN_SiOVERNIGHT"].fillna(0)

In [ ]:
# SAVE THE PARQUET IN THE DATA FOLDER FOR EASY ACCESS NEXT TIME

df.to_parquet("../../data/high_10m.parquet", index = False)